<a href="https://colab.research.google.com/github/jfcaro/Vacunacion_COVID_19_ESP/blob/main/Ritmo_vacunacion_Covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ritmo de vacunación España (14/02/2021)
 Cálculo del % de población por comunidad vacunada (pauta completa)<br>
 Cálculo de fecha para conseguir 100% de vacuación manteniendo velocidad actual <br>
 Cálculo de la velocidad diaria para conseguir vacunación del 70% de población <br>
 
 @jfrcaro

In [ ]:
#Instalar liberías

# odfpy necesario para tratar hojas de cálculo en formato odf
!pip install odfpy
# !pip install pandas
# !pip install datetime

# importar librerías 
import pandas as pd
from datetime import datetime, timedelta

### ¿Cuál es la población por comunidad?
Para la obtención de la población por comunidad se emplea una tabla de la wikipedia : https://es.wikipedia.org/wiki/Comunidad_aut%C3%B3noma

In [ ]:
# Obtencion carga de las tablas de la página https://es.wikipedia.org/wiki/Comunidad_aut%C3%B3noma'
comunidades = pd.read_html('https://es.wikipedia.org/wiki/Comunidad_aut%C3%B3noma')

In [ ]:
# Obtención de tabla con población por comunidad sin totales, incluyendo ajuste de valores

# Seleccionamos la tabla 5 de wikipedia que es la relacionada con población
comunidad = comunidades[4]

# Ajustes diversos en el campo Población(hab.)
comunidad["Población(hab.)"] = comunidad["Población(hab.)"].astype(str)
comunidad["Población(hab.)"]= comunidad["Población(hab.)"].apply(lambda x: str(x).replace(' ', ''))
comunidad["Población(hab.)"]= comunidad["Población(hab.)"].apply(lambda x: str(x).replace(u'\xa0', u''))

# Mostrar tabla 
comunidad

,Posición,Nombre,Población(hab.),%,Densidad(hab./km²)
0,1,Andalucía,8379248,1799,9602
1,2,Cataluña,7596131,1622,23533
2,3,Comunidad de Madrid,6576009,1397,81117
3,4,Comunidad Valenciana,4959243,1061,21249
4,5,Galicia,2700970,582,9158
5,6,Castilla y León,2407650,521,2574
6,7,País Vasco,2198657,471,30331
7,8,Canarias,2126779,451,28308
8,9,Castilla-La Mancha,2025510,453,2662
9,10,Región de Murcia,1477946,316,12996


### ¿Cuáles son los últimos datos sobre vacunación? 
Para la obtención de las estadísticas de vacunación se emplea tabla de https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm

In [ ]:
# Obtención de tabla de vacunación. Viene en formato ods por lo que se requiere importar odfpy

# Obtención del día actual
hoy = datetime.today()

# Día de la semana de hoy
diasemana = hoy.weekday()

# Se coge el día anterior para asegurar que existe el fichero, en el caso de fin de semana y lunes se toma viernes
if diasemana in [1,2,3,4,5]:
    hoy = datetime.today()-timedelta(days = 1)
elif diasemana == 0: 
    hoy = datetime.today()-timedelta(days = 3)
elif diasemana == 6:
    hoy = datetime.today()-timedelta(days = 2)

# Función para convertir mes y día a dos cifras
def conversion(numero):
    if numero >= 10: 
        return str(numero)
    else:
        return "0"+str(numero)
    
# Nombre del fichero a descargar desde www.mscbs.gob.es 
nombre_fichero = "Informe_Comunicacion_"+str(hoy.year)+ conversion(hoy.month)+ conversion(hoy.day)+".ods"

# descargar el fichero de hoy en formato odf para lo que hay que instalar odfpy
try:     
    vacunacion = pd.read_excel("https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/"+nombre_fichero, engine="odf")
except:
    print ("No se puede cargar el fichero: Verificar que existe {}".format("https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/"+ nombre_fichero))

In [ ]:
# Cambio de nombre de columna Unnamed  por Comunidad
vacunacion.rename(columns={'Unnamed: 0':'Comunidad'}, inplace=True)

### ¿Cómo fusionar las tablas?
Para fusionar las tablas es necesario encontrar un campo común en este caso en nombre de la comunidad <br>
A continuación se realizan algunas modificaciones para poder fusionar

In [ ]:
# Cambiar del dataframe vacunación el nombre de algunas comunidades para igualar al de comunidad y poder fusionar
vacunacion.loc[15, 'Comunidad'] = "Comunidad Foral de Navarra"
vacunacion.loc[9, 'Comunidad'] = "Comunidad Valenciana"
vacunacion.loc[13, 'Comunidad'] = "Comunidad de Madrid"
vacunacion.loc[6, 'Comunidad'] = "Castilla y León"
vacunacion.loc[7, 'Comunidad'] = "Castilla-La Mancha"
vacunacion.loc[3, 'Comunidad'] = "Islas Baleares"
vacunacion.loc[2, 'Comunidad'] = "Principado de Asturias"
vacunacion.loc[14, 'Comunidad'] = "Región de Murcia"
vacunacion.loc[19, 'Comunidad'] = "TOTAL"

In [ ]:
# % sobre entregadas en tanto por 100
vacunacion['% sobre entregadas']=vacunacion['% sobre entregadas']*100

# añadir fecha media en la fila de totales
vacunacion.loc[19, "Fecha de la última vacuna registrada (2)"]= vacunacion["Fecha de la última vacuna registrada (2)"].mean()

In [ ]:
# Mostar la table de vacunación
vacunacion

,Comunidad,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,475110,36300,34500,545910,431642,7906.834460,176731,2021-02-11
1,Aragón,81045,6300,5400,92745,88297,9520.405413,35659,2021-02-11
2,Principado de Asturias,83975,6300,4200,94475,78446,8303.360677,32946,2021-02-11
3,Islas Baleares,42480,3200,4800,50480,40624,8047.543582,16683,2021-02-11
4,Canarias,93570,7000,8900,109470,99316,9072.439938,40779,2021-02-11
5,Cantabria,39345,2900,2400,44645,39707,8893.941091,17478,2021-02-11
6,Castilla y León,177555,13500,9800,200855,178831,8903.487591,75433,2021-02-11
7,Castilla-La Mancha,120375,9200,8300,137875,117671,8534.614687,41690,2021-02-11
8,Cataluña,410120,31400,31800,473320,376519,7954.850841,152091,2021-02-11
9,Comunidad Valenciana,207280,15700,20600,243580,214010,8786.025125,92433,2021-02-11


In [ ]:
# Fusión tabla vacunacion y comunidad a través del nombre de la comunidad 
completa = vacunacion.merge(comunidad, left_on='Comunidad', right_on='Nombre')

### Cálculo de indicadores
%vacunado : Es el porcentaje de vacunados (pauta completa) <br><br>
fecha_restante: Es la fecha para completar el 100% al ritmo actual que son los vacunados actuales en el tiempo desde inicio de vacunación (27/12-2020)<br>
+ info: https://elpais.com/sociedad/2020-12-18/la-vacunacion-en-espana-arrancara-el-27-de-diciembre.html <br>

ritmo (vacunaciones completas por día) 70%: es el ritmo en vacunaciones pauta completa por día a aplicar desde el día posterior al cálculo para llegar <br>
al objetivo del 70% de la población al final del verano (21/09/2021)
+ info: https://www.eldiario.es/politica/pedro-sanchez-defiende-proceso-vacunacion-confia-llegara-70-poblacion-verano_1_7036447.html
+ info: https://m.europapress.es/eseuropa/noticia-comision-europea-von-der-leyen-dice-fecha-limite-vacunar-70-poblacion-adulta-21-septiembre-20210212153122.html


In [ ]:
# Se añade columna para % vacunación por comunidad 
completa['Nº Personas vacunadas(pauta completada)']= completa['Nº Personas vacunadas(pauta completada)'].astype(int)
completa['Población(hab.)']= completa['Población(hab.)'].astype(int)
completa['%vacunado'] = (completa['Nº Personas vacunadas(pauta completada)']/completa['Población(hab.)'])*100

In [ ]:
# Se añade columna tiempo estimado restante en función de velocidad desde inicio de vacunación (27-12-2020)
fecha_inicio_vacunacion = datetime(2020,12,27)
completa['dias_desde_inicio'] =  completa['Fecha de la última vacuna registrada (2)'].apply(lambda x: ((x - fecha_inicio_vacunacion).days))
completa['dias_restantes'] =completa['dias_desde_inicio']/(completa['%vacunado']/100)
completa['fecha_restante'] =completa['dias_restantes'].apply(lambda x:(hoy + timedelta(days = x)).strftime('%d/%m/%Y'))                                                         

In [ ]:
# Se añade velocidad en personas vacunadas(pauta completa) al día para llegar al 70% el 21 de Septiembre de 2021
verano = datetime(2021,9,21)
dias_hasta_verano = (verano - datetime.today()).days
completa['ritmo (vacunaciones completas por día) 70%']=(completa['Población(hab.)']*0.7-completa['Nº Personas vacunadas(pauta completada)'])/dias_hasta_verano

In [ ]:
# Columnas a incluir en la visualización final 
columnas_vacunacion = list(vacunacion.columns)
columnas_poblacion = ['Población(hab.)']
columnas_nuevas = ['%vacunado', 'fecha_restante', 'ritmo (vacunaciones completas por día) 70%']
total_columnas = columnas_vacunacion + columnas_poblacion + columnas_nuevas

In [ ]:
# Visualizar Dataframe
completa[total_columnas]

,Comunidad,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2),Población(hab.),%vacunado,fecha_restante,ritmo (vacunaciones completas por día) 70%
0,Andalucía,475110,36300,34500,545910,431642,7906.834460,176731,2021-02-11,8379248,2.109151,02/02/2027,26095.149541
1,Aragón,81045,6300,5400,92745,88297,9520.405413,35659,2021-02-11,1307984,2.726257,27/09/2025,4036.375229
2,Principado de Asturias,83975,6300,4200,94475,78446,8303.360677,32946,2021-02-11,1028135,3.204443,18/01/2025,3150.222477
3,Islas Baleares,42480,3200,4800,50480,40624,8047.543582,16683,2021-02-11,1128139,1.478807,20/08/2029,3545.937156
4,Canarias,93570,7000,8900,109470,99316,9072.439938,40779,2021-02-11,2126779,1.917407,08/09/2027,6642.047248
5,Cantabria,39345,2900,2400,44645,39707,8893.941091,17478,2021-02-11,580067,3.013100,19/04/2025,1782.426147
6,Castilla y León,177555,13500,9800,200855,178831,8903.487591,75433,2021-02-11,2407650,3.133055,20/02/2025,7384.963303
7,Castilla-La Mancha,120375,9200,8300,137875,117671,8534.614687,41690,2021-02-11,2025510,2.058247,28/03/2027,6312.692661
8,Cataluña,410120,31400,31800,473320,376519,7954.850841,152091,2021-02-11,7596131,2.002217,30/05/2027,23693.581193
9,Comunidad Valenciana,207280,15700,20600,243580,214010,8786.025125,92433,2021-02-11,4959243,1.863853,16/11/2027,15500.170183


### Se muestran en pantalla los indicadores calculados

In [ ]:
# Indicadores para España
print ("A día de {} se ha vacunado (pauta completa) al {:,.2f}% de la población española.".format(str(hoy.strftime('%d/%m/%Y')), float(completa["%vacunado"][completa['Comunidad']=='TOTAL'].values[0])))
print ("El ritmo de vacunaciones en España desde el {} ha de ser de {:,.2f} vacunaciones/día para vacunar al 70% de la población a 21/09/2021.".format(str(hoy.strftime('%d/%m/%Y')), completa["ritmo (vacunaciones completas por día) 70%"][completa['Comunidad']=='TOTAL'].values[0]))

personas_pautacompleta = float(completa["Nº Personas vacunadas(pauta completada)"][completa['Comunidad']=='TOTAL'].values[0])
dias_desde_inicio = completa['dias_desde_inicio'][completa['Comunidad']=='TOTAL'].values[0] 
personas_por_dia= personas_pautacompleta/dias_desde_inicio

print ("Desde el inicio de la vacunación van {} días y se han vacunado (pauta completa) a {:,.0f} personas, lo que hace una media de {:,.2f} vacuanados por día.".format(dias_desde_inicio, personas_pautacompleta, personas_por_dia))
print ("Al ritmo anterior de vacunados por día se cubrirá el 100% de la población el {}.".format(completa["fecha_restante"][completa['Comunidad']=='TOTAL'].values[0]))

A día de 12/02/2021 se ha vacunado (pauta completa) al 2.14% de la población española.
El ritmo de vacunaciones en España desde el 12/02/2021 ha de ser de 145,360.15 vacunaciones/día para vacunar al 70% de la población a 21/09/2021.
Desde el inicio de la vacunación van 46 días y se han vacunado (pauta completa) a 1,000,485 personas, lo que hace una media de 21,749.67 vacuanados por día.
Al ritmo anterior de vacunados por día se cubrirá el 100% de la población el 30/12/2026.


In [ ]:
# Indicadores por comunidad 

# Se obtienen las comunidades en lista y se le quita la fila de totales
comunidades = sorted(completa['Comunidad'].to_list()[0:-1])

In [ ]:
# Se recorren todas las comunidades
for comunidad in comunidades: 
    print ("** DATOS DE LA COMUNIDAD AUTÓNOMA: {}".format(comunidad.upper()))
    print ("A día de {} se ha vacunado (pauta completa) al {:,.2f}% de la población de la comunidad.".format(str(hoy.strftime('%d/%m/%Y')), float(completa["%vacunado"][completa['Comunidad']==comunidad].values[0])))
    print ("El ritmo de vacunaciones la comunidad desde el {} ha de ser de {:,.2f} vacunaciones/día para vacunar al 70% de su población a 21/09/2021.".format(str(hoy.strftime('%d/%m/%Y')), completa["ritmo (vacunaciones completas por día) 70%"][completa['Comunidad']==comunidad].values[0]))

    personas_pautacompleta = float(completa["Nº Personas vacunadas(pauta completada)"][completa['Comunidad']==comunidad].values[0])
    dias_desde_inicio = completa['dias_desde_inicio'][completa['Comunidad']==comunidad].values[0] 
    personas_por_dia= personas_pautacompleta/dias_desde_inicio

    print ("Desde el inicio de la vacunación van {} días y se han vacunado (pauta completa) a {:,.0f} personas, lo que hace una media de {:,.2f} vacuanados por día.".format(dias_desde_inicio, personas_pautacompleta, personas_por_dia))
    print ("Al ritmo anterior de vacunados por día se cubrirá el 100% de su población el {}.".format(completa["fecha_restante"][completa['Comunidad']==comunidad].values[0]))
    print ("\n")
   

** DATOS DE LA COMUNIDAD AUTÓNOMA: ANDALUCÍA
A día de 12/02/2021 se ha vacunado (pauta completa) al 2.11% de la población de la comunidad.
El ritmo de vacunaciones la comunidad desde el 12/02/2021 ha de ser de 26,095.15 vacunaciones/día para vacunar al 70% de su población a 21/09/2021.
Desde el inicio de la vacunación van 46 días y se han vacunado (pauta completa) a 176,731 personas, lo que hace una media de 3,841.98 vacuanados por día.
Al ritmo anterior de vacunados por día se cubrirá el 100% de su población el 02/02/2027.


** DATOS DE LA COMUNIDAD AUTÓNOMA: ARAGÓN
A día de 12/02/2021 se ha vacunado (pauta completa) al 2.73% de la población de la comunidad.
El ritmo de vacunaciones la comunidad desde el 12/02/2021 ha de ser de 4,036.38 vacunaciones/día para vacunar al 70% de su población a 21/09/2021.
Desde el inicio de la vacunación van 46 días y se han vacunado (pauta completa) a 35,659 personas, lo que hace una media de 775.20 vacuanados por día.
Al ritmo anterior de vacunados por

### ¿Quieres la tabla en excel?

In [ ]:
# Exporta dataframe completo a excel
completa[total_columnas].to_excel('./Vacunacion.xlsx', 'Datos',index = False)

### Para continuar
Afinar el cálculo del 100% de la población se podría calcular la última velocidad conseguida como vacunados día n - vacunados día (n-1) y aplicar ese ritmo al <br>
cálculo de fecha restante<br><br>
Obtener todas tablas generadas en https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm y ver la evolución temporal <br> 
de los indicadores<br><br>
Identificar concepto de "Población adulta" y obtención de la población adulta por comunidad para recalcular el objetivo al 70% de la población adulta

